In [ ]:
sample_size_to_animate_on = 5 #this has to be less than or equal to the sample_size
epsilon = 0.1
hypothesis_class_cardinality = 10000
interpolation_mode = "continuous"

highlighted_hypothesis = 5 #[0-7]
find_points = True

In [ ]:
data_path = '/content/drive/My Drive/Simulations in Stat learning/data/vc_choreo/'
distributions_path = '/content/drive/My Drive/Simulations in Stat learning/distributions/'
videos_path = '/content/drive/My Drive/Simulations in Stat learning/videos/vc_choreo/'

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib
%matplotlib inline
from scipy.stats import multivariate_normal
import pickle
import os

In [ ]:
sample_size = sample_size_to_animate_on
true_unknown_distribution = "circle"
is_agnostic = False
specific_hypothesis = highlighted_hypothesis




In [ ]:
np.random.seed(42)
#Standard
plt.rcParams["figure.figsize"] = (24,18)
plt.rcParams['axes.linewidth'] = 7
plt.rcParams['savefig.facecolor']='blue'
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 
plt.rcParams['xtick.major.pad']='25'
plt.rcParams['ytick.major.pad']='25'

# Supporting Functions

In [ ]:
#CUSTOM COLORMAPS

label_colors = matplotlib.colors.ListedColormap(['silver', 'Black'])
right_wrong_colors = matplotlib.colors.ListedColormap(['tomato', 'forestgreen'])
hypothesis_colors = matplotlib.colors.ListedColormap(['tomato', 'silver'])


In [ ]:
#norm = colors.Normalize(0, 1)
def get_facecolors(array,mode,alpha = 1):
    shape = array.shape
    p = (-1,)
    if mode == "greenred":
        colors = right_wrong_colors(array.reshape(-1),alpha)
    if mode == "black":
        
        colors = cm.binary(array.reshape(-1),alpha)
        
    return colors.reshape(shape + p)

In [ ]:
#This get labels is slightly different from the one in hoeffdings inequality notebook
def get_labels(x,y,distribution):
    if distribution == "circle":
        z = np.where(x**2 + y**2 + 2*0.9*x + 2*0.5*y - 0.2 > 0 , 1 , 0)  
        
    elif distribution == "linear":
        h_labeling = np.array([0.7,0.5])
        
        temp = np.c_[x,y]   #---for animation
        z = np.where(np.matmul(temp,h_labeling) - 0> 0.0, 1, 0)
        
    elif distribution =="wiggly1":
        z = np.where(3*y**3 +x**2 -0.2*y**2 +  0*x - 0.01   >0, 1, 0)
        
    elif distribution =="wiggly2":
        z = np.where(30*x**10 + 2*y**10 -700*x**7 + 20*y**5 -1*x**2 + 0.02   >0, 0, 1)
        
    elif distribution == "l1":
        z1 = np.where(abs(x)+abs(y)>0.15,1,0)
        z2 = np.where(abs(x)+abs(y)>0.4,1,0)

        z = np.logical_xor(z1,z2)
    
    else :
        filename = distributions_path + distribution + ".png"
        im = plt.imread(filename)
        image = np.round(np.sum(im,2)/3)

        x_ = np.floor((x + 0.4)*1000)
        y_ = np.floor((y + 0.4)*1000)

        labels = []
        for i,j in zip(x_.astype(int),y_.astype(int)):

            label = image[i,j]
            labels.append(label)

        z = np.abs(1 - np.array(labels))
    
    
        
    return z

In [ ]:
def errors_on_hypothesis_set(data,hypothesis):
    intermediate = np.matmul(data[:,:2], hypothesis)
    predictions = np.where(intermediate > 0.0, 1, 0)
    labels = data[:,2]

    E_out = np.logical_xor(predictions.T,labels)
    true_errors = 1 - np.sum(E_out,axis=1)/E_out.shape[1]
    
    return true_errors,E_out,predictions  #this depends on which all hypothesis I want to animate on

__________________________________________________________________________________________________________________

# All the knobs

In [ ]:
#PAC LEARNING
#All the knobs here
n = sample_size_to_animate_on
distribution = true_unknown_distribution
agnostic = is_agnostic
#interpolation_type = 

if agnostic:
  name = "agnostic"
else:
  name = distribution



e = epsilon
vc_dim = 2



#THE HYPOTHESIS SPACE
num_hypothesis = hypothesis_class_cardinality
hypothesis = np.random.uniform(-1,1,[2,num_hypothesis])
h8 = np.array([[0.5,0.5],
               [0.5,0],
               [0.5,-0.5],
               [-0.5,0.5],
               [-0.5,0],
               [-0.5,-0.5],
               [0,0.5],
               [0,-0.5]])
hypothesis[:,:8] = h8.T




data = np.load(data_path + "data_"+name+'_'+str(40000)+".npy")
training_data = np.load(data_path + "training_data_"+name+".npy")







# do big matrix multiplications


In [ ]:

true_errors,E_out,p_true = errors_on_hypothesis_set(data,hypothesis)
in_sample_errors,E_in,_ = errors_on_hypothesis_set(training_data,hypothesis)

In [ ]:
h_erm = highlighted_hypothesis#np.argmin(in_sample_errors)  #this is an artifact and confusing

In [ ]:
#Taking a slice for the current hypothesis. First 8 are going to be SPECIFIC hypothesis
E_out = E_out[h_erm,:]
p_true = p_true[:,h_erm]

___________________________________________________________________________________________________________________________

# Plotting

In [ ]:
#Formatting the plot
fig, axs = plt.subplots(1, 2)
plt.autoscale(False)

axs[0].set_xlim([-0.4, 0.4])
axs[0].set_ylim([-0.4, 0.4])
axs[0].set_aspect(aspect='equal', adjustable='box')

axs[1].set_xlim([-1, 1])
axs[1].set_ylim([-1, 1])
axs[1].set_aspect(aspect='equal', adjustable='box')






#When animating the hypothesis I need to have a reference over this
data_labels = axs[0].scatter(data[:,0],data[:,1],c=right_wrong_colors(E_out,0.02),s = 20)# replace 0.02 with 1
training_data_labels  = axs[0].scatter(training_data[:,0],training_data[:,1],c=right_wrong_colors(E_in[h_erm,:],1),s = 250) #animate these

#diff = np.abs(true_errors - in_sample_errors)
#hypothesis_space_2 = axs[1].scatter(hypothesis[0,:],hypothesis[1,:],c = hypothesis_colors(diff<e,1),s=400)

hypothesis_space_2 = axs[1].scatter(hypothesis[0,:],hypothesis[1,:],c = get_facecolors(in_sample_errors,mode="black"),s=400)

#For Animating Current hypothesis
x = hypothesis[0,h_erm]
y = hypothesis[1,h_erm]

current_hypothesis_0 = axs[1].scatter(x,y,s=800,color = 'black')
current_color = axs[1].scatter(x,y,s=400,color = 'silver')




#fig.colorbar(hypothesis_space_0,ax=axs[1],fraction=0.046, pad=0.1)
#fig.colorbar(hypothesis_space_1,ax=axs[2],fraction=0.046, pad=0.1)
#fig.colorbar(hypothesis_space_2,ax=axs[3],fraction=0.046, pad=0.1)



# Just moving in the sample space

In [ ]:
def get_linear_interpolated_points(current_points,final_points,frames):
    shape = current_points.shape
    diff = final_points.reshape(-1) - current_points.reshape(-1)
    offset = diff/frames
    interpolated_points = []
    for i in range(frames):
        points_n = current_points.reshape(-1) + offset*i
        interpolated_points.append(points_n.reshape(shape))
    return np.array(interpolated_points).T

In [ ]:
x

In [ ]:
y

In [ ]:
frames = 91

multiplier = int(1/((frames - 1)/360))


radius = np.sqrt(x*x+y*y)
theta_x = np.rad2deg(np.arccos(x/radius))
theta_y = np.rad2deg(np.arcsin(y/radius))

circle_hypothesis = []

for i in range(frames):
  new_x = radius* np.cos(np.deg2rad(theta_x + multiplier*i))
  new_y = radius* np.sin(np.deg2rad(theta_y + multiplier*i))
  circle_hypothesis.append(np.array([new_x,new_y]))
   

In [ ]:
infinite_hypothesis = np.array(circle_hypothesis).T

In [ ]:
infinite_hypothesis[:,0]

In [ ]:
_,E_out,_ = errors_on_hypothesis_set(data,infinite_hypothesis)
_,E_in,_ = errors_on_hypothesis_set(training_data,infinite_hypothesis)

In [ ]:
colors_background = []
colors_sample = []

for i in range(frames):
  colors_background.append(right_wrong_colors(E_out[i,:],0.02))
  colors_sample.append(right_wrong_colors(E_in[i,:],1))

colors_background = np.array(colors_background)
colors_sample = np.array(colors_sample)

In [ ]:
finite_hypothesis = infinite_hypothesis

In [ ]:
finite_hypothesis.shape

# Animation

In [ ]:
def animate(i,colors_background,colors_sample,finite_hypothesis):
    
    data_labels.set_facecolors(colors_background[i,:,:])
    training_data_labels.set_facecolors(colors_sample[i,:,:])

    current_color.set_offsets(finite_hypothesis[:,i])
    current_hypothesis_0.set_offsets(finite_hypothesis[:,i])

    
    return data_labels,training_data_labels,current_hypothesis_0,current_color,

In [ ]:
name = "mhypothesis_"+name

In [ ]:
ani = animation.FuncAnimation(fig, animate, fargs=(colors_background,colors_sample,finite_hypothesis,), 
            frames=frames, interval=10, repeat=False,blit = True) 

ani.save(videos_path+ name +'_'+str(sample_size_to_animate_on)+'.mp4', fps=30, extra_args=['-vcodec', 'libx264'],savefig_kwargs=dict(facecolor='blue'))